In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import NoSuchElementException
import time
import string
import openpyxl
import os
import json


In [2]:
#Loading Selenium webdriver

In [3]:
PATH = ("C:\\Users\\fatih\\Downloads\\chromedriver_win32\\chromedriver.exe")
driver = webdriver.Chrome(PATH)


<ipython-input-3-b6bf87c95fd0>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [4]:
UoT= 'https://www.google.ca/maps/place/University+of+Toronto+-+St.+George+Campus/@43.6628917,-79.3978504,17z/data=!4m7!3m6!1s0x882b34b8f331fd9b:0x8d1d9bb6765a76f7!8m2!3d43.6628917!4d-79.3956564!9m1!1b1'
ReU= 'https://www.google.com/maps/place/Ryerson+University/@43.6576585,-79.3788017,17z/data=!4m7!3m6!1s0x89d4cb35431c1395:0xe8ed8bd69125d6f4!8m2!3d43.6576585!4d-79.3788017!9m1!1b1'
WeU= 'https://www.google.ca/maps/place/Western+University/@43.0095971,-81.2759223,17z/data=!4m7!3m6!1s0x882eee0e6ac42bd1:0xaa87f353aab9799b!8m2!3d43.0095971!4d-81.2737336!9m1!1b1'
UoO= 'https://www.google.ca/maps/place/University+of+Ottawa/@45.4231064,-75.6853216,17z/data=!4m7!3m6!1s0x4cce050a6db98d73:0x188a59c3622fdbae!8m2!3d45.4231064!4d-75.6831329!9m1!1b1'
YoU= 'https://www.google.ca/maps/place/York+University/@43.7734535,-79.5040571,17z/data=!4m7!3m6!1s0x89d4cd330b767bfb:0xdbb899cc9da76d19!8m2!3d43.7734535!4d-79.5018684!9m1!1b1'


In [5]:
#Opening Google maps url at a time & add sleeping duration to allow for full loading  
driver.get(WeU)
time.sleep(3)

In [6]:
#Creating a scrollable dynamic function to show all the reviews 
scrollable_div_WeU = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
for i in range(0,10):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div_WeU)
    time.sleep(1)

<ipython-input-6-b6dc058880eb>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scrollable_div_WeU = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')


In [7]:
import numpy as np
from Crypto.PublicKey import RSA
import json
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import os 
import requests
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
%matplotlib inline

In [8]:
#parsing the webpage
response_WeU = bs(driver.page_source, 'html.parser')
#closing the dirver after parsing it to start building and analysing the outcome
driver.close()

In [9]:
reviews_WeU = response_WeU.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
len(reviews_WeU)

100

In [10]:
# creating a review summary
# Select the items that will be used in the analysis
def get_review_summary(result_set):
    rev_dict = {'Review Rate': [], 'Review Time': [], 'Review Text': [], 'User': [], 'Impact': []}
    for result in result_set:
        review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
        review_time = result.find('span', class_='ODSEW-ShBeI-RgZmSc-date').text
        review_text = result.find('span', class_='ODSEW-ShBeI-text').text
        reviewer = result.find('div', class_='ODSEW-ShBeI-title').text
        Usefulness = result.find('div', class_='ODSEW-ShBeI-VdSJob').text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)
        rev_dict['User'].append(reviewer)
        rev_dict['Impact'].append(Usefulness)

    return(pd.DataFrame(rev_dict))

rev_summ = get_review_summary(reviews_WeU)
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,6 days ago,"Emergency staff, nurses and doctors were all v...",Sam Bosma,Local Guide4 reviews
1,5 stars,4 months ago,Love being part of such incredible history. So...,Omran Sheikh,Local Guide · 204 reviews
2,1 star,2 months ago,I am ashamed of what was my own university for...,Marion Gerull,Local Guide1 review
3,4 stars,a year ago,Seems like a nice looking campus. We came here...,S G,Local Guide · 194 reviews
4,5 stars,2 weeks ago,I had a double bypass. Every person from the ...,Rennie Dykens,Local Guide1 review
...,...,...,...,...,...
95,2 stars,2 years ago,"2/10 rating give to this university, first, th...",刘缀之,Local Guide · 26 reviews
96,1 star,5 years ago,A few friends now have told me that they can't...,Babi Zee,Local Guide12 reviews
97,1 star,3 years ago,"Lost a $5,000 scholarship because my departmen...",Michael Boyle,Local Guide · 89 reviews
98,5 stars,5 years ago,I had the time of my life at the university. I...,Hu Yun Lo,Local Guide · 583 reviews


# data cleaning

In [11]:
rev_summ['Impact'] = rev_summ['Impact'].map(lambda x: x.lstrip('Local Guide ·'))

In [12]:
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,6 days ago,"Emergency staff, nurses and doctors were all v...",Sam Bosma,4 reviews
1,5 stars,4 months ago,Love being part of such incredible history. So...,Omran Sheikh,204 reviews
2,1 star,2 months ago,I am ashamed of what was my own university for...,Marion Gerull,1 review
3,4 stars,a year ago,Seems like a nice looking campus. We came here...,S G,194 reviews
4,5 stars,2 weeks ago,I had a double bypass. Every person from the ...,Rennie Dykens,1 review
...,...,...,...,...,...
95,2 stars,2 years ago,"2/10 rating give to this university, first, th...",刘缀之,26 reviews
96,1 star,5 years ago,A few friends now have told me that they can't...,Babi Zee,12 reviews
97,1 star,3 years ago,"Lost a $5,000 scholarship because my departmen...",Michael Boyle,89 reviews
98,5 stars,5 years ago,I had the time of my life at the university. I...,Hu Yun Lo,583 reviews


In [13]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
stop = stopwords.words('english')
rev_summ['Review Text'] = rev_summ['Review Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
rev_summ

,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,6 days ago,"Emergency staff, nurses doctors friendly helpf...",Sam Bosma,4 reviews
1,5 stars,4 months ago,Love part incredible history. So proud kid. #W...,Omran Sheikh,204 reviews
2,1 star,2 months ago,I ashamed university forcing ethics professor ...,Marion Gerull,1 review
3,4 stars,a year ago,Seems like nice looking campus. We came confer...,S G,194 reviews
4,5 stars,2 weeks ago,"I double bypass. Every person surgeon, nurses,...",Rennie Dykens,1 review
...,...,...,...,...,...
95,2 stars,2 years ago,"2/10 rating give university, first, campus bea...",刘缀之,26 reviews
96,1 star,5 years ago,A friends told can't get jobs there; they're s...,Babi Zee,12 reviews
97,1 star,3 years ago,"Lost $5,000 scholarship department counsellor ...",Michael Boyle,89 reviews
98,5 stars,5 years ago,I time life university. I'm kind sad calling '...,Hu Yun Lo,583 reviews


In [16]:
description_full = rev_summ.set_index('Review Text')
description_full.head(5)

,Review Rate,Review Time,User,Impact
Review Text,,,,
"Emergency staff, nurses doctors friendly helpful made stay enjoyable.",5 stars,6 days ago,Sam Bosma,4 reviews
Love part incredible history. So proud kid. #WESTERNDAD,5 stars,4 months ago,Omran Sheikh,204 reviews
I ashamed university forcing ethics professor position Huron College teaching loving exact thing teaching students faithfully two decades. Where institution gone …,1 star,2 months ago,Marion Gerull,1 review
"Seems like nice looking campus. We came conference enjoyed walking around property, many students around early summer.",4 stars,a year ago,S G,194 reviews
"I double bypass. Every person surgeon, nurses, p.s.w.'s, room cleaners professional personable treated kindness respect. Thank all!",5 stars,2 weeks ago,Rennie Dykens,1 review


In [17]:
rev_summ['Review Text'] = rev_summ['Review Text'].str.replace('[^\w\s]',' ')
rev_summ

<ipython-input-17-3283235791aa>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  rev_summ['Review Text'] = rev_summ['Review Text'].str.replace('[^\w\s]',' ')


,Review Rate,Review Time,Review Text,User,Impact
0,5 stars,6 days ago,Emergency staff nurses doctors friendly helpf...,Sam Bosma,4 reviews
1,5 stars,4 months ago,Love part incredible history So proud kid W...,Omran Sheikh,204 reviews
2,1 star,2 months ago,I ashamed university forcing ethics professor ...,Marion Gerull,1 review
3,4 stars,a year ago,Seems like nice looking campus We came confer...,S G,194 reviews
4,5 stars,2 weeks ago,I double bypass Every person surgeon nurses ...,Rennie Dykens,1 review
...,...,...,...,...,...
95,2 stars,2 years ago,2 10 rating give university first campus bea...,刘缀之,26 reviews
96,1 star,5 years ago,A friends told can t get jobs there they re s...,Babi Zee,12 reviews
97,1 star,3 years ago,Lost 5 000 scholarship department counsellor ...,Michael Boyle,89 reviews
98,5 stars,5 years ago,I time life university I m kind sad calling ...,Hu Yun Lo,583 reviews


In [18]:
rev_summ.to_csv("reviews_WeU.csv")